Model Setup
-----------

2D, Heat Equation with Dirichlet BC at top and bottom boundary.  No variation in the x direction.

\\[
\frac{\partial T}{\partial t} = -v \cdot \frac{\partial T}{\partial y} +k \frac{\partial^2 T}{\partial y^2}+H
\\]

\\[
\frac{\partial T}{\partial x} = 0
\\]

with $0 \leqslant x \leqslant 1 $ and $ y_{0}\leqslant y \leqslant y_{1}$

$T(y_{1}) = T_{1}$

$ -k\nabla{T}\mid_{y_{0}} = \left[\,0.0,\,f\,\right] $

------

Effectively a 1D problem in $y$-axis, described by the analytic function

$c_{0} = ( \frac{f}{k}-\frac{h}{v} ) \cdot \frac{k}{v} \cdot \exp \left[-\frac{v}{k}y_{0} \right]$

$c_{1} = T_{1}-c_{0}\exp \left[\frac{v}{k}y_{1} \right] - \frac{h}{v}y_{1}$


$T = c_{0} \exp \left[\frac{v}{k}y\right] + \frac{h}{v}y + c_{1}$

In [ ]:
import underworld as uw
import glucifer
import numpy as np

# for machines without matplotlib #
make_graphs = True
try:
    import matplotlib
except ImportError:
    make_graphs=False

# depth range
y0 = -0.60
y1 = 1.3

T1 = 8.0       # surface temperature
k = 0.70       # diffusivity
h = 9.0        # heat production, source term
f = -5.23      # heat flux vector
v = 0.3        # velocity

# analytic solution definitions
c0 = (-f/k-h/v) * k/v * np.exp(-v/k*y0)
c1 = T1 - c0*np.exp(v/k*y1) - h/v*y1

def analyticT(y):
    return c0*np.exp(v/k*y) + h/v*y + c1

def analytic_dT_dy(y):
    return v/k*c0*np.exp(v/k*y) + h/v

In [ ]:
# without velocity
# c0 = (-f[1]+h*y0)/k
# c1 = T1 + h/(2.0*k)*y1*y1 - c0*y1

# # # analytic solution definitions
# def analyticT(y):
#      return -h/(2*k)*y*y + c0*y + c1

# def analytic_dT_dy(y):
#     return -h/k*y + c0

In [ ]:
# build mesh and fields
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1"), 
                                 elementRes  = (10, 10), 
                                 minCoord    = (0., y0), 
                                 maxCoord    = (1., y1))

tField    = uw.mesh.MeshVariable(mesh=mesh, nodeDofCount=1, dataType='double')
tDotField = uw.mesh.MeshVariable(mesh=mesh, nodeDofCount=1, dataType='double')
vField    = uw.mesh.MeshVariable(mesh=mesh, nodeDofCount=2, dataType='double')

# set entire tField to T1
tField.data[:] = T1

# set constant velocity field
vField.data[:] = (0.0,v)

# define neumann condition - flux!
nbc = uw.conditions.NeumannCondition( flux=[0.0,f], 
                                      variable=tField, 
                                      nodeIndexSet=(mesh.specialSets['MinJ_VertexSet']) )

# flag top boundary nodes with dirichlet conditions
bc = uw.conditions.DirichletCondition(tField, indexSetsPerDof=(mesh.specialSets['MaxJ_VertexSet']) )

# define heat eq. system
ss = uw.systems.AdvectionDiffusion( phiField       = tField,
                                    phiDotField    = tDotField,
                                    velocityField  = vField,
                                    fn_diffusivity = k,
                                    fn_sourceTerm  = h,
                                    conditions     = [bc, nbc] ) 

In [ ]:
# evolve to a steady state, ie, < 1e-4 relative variation
er = 1.0
its = 0              # iteration count
tOld = tField.copy() # old temperature field

while er > 1e-5:
    
    tOld.data[:] = tField.data[:] # record old values
    
    dt = ss.get_max_dt()          # get time steps
    ss.integrate(dt)              # integrate in time (solve)
    
    absErr = uw.utils._nps_2norm(tOld.data-tField.data)
    magT   = uw.utils._nps_2norm(tOld.data)
    er = absErr/magT              # calculate relative variation
    
    its += 1

print "Number of iterations ",its


In [ ]:
numeric     = np.zeros(len(mesh.specialSets['MinI_VertexSet'].data))
ycoord      = np.zeros_like(numeric)
analytic    = np.zeros_like(numeric)

ids = mesh.specialSets['MinI_VertexSet']
numeric[:] = tField.evaluate(ids).T

ycoord = tField.mesh.data[ids.data,[1]]
analytic = analyticT(ycoord)

abserr = uw.utils._nps_2norm(analytic - numeric)
mag = uw.utils._nps_2norm(analytic)
relerr = abserr / mag

from mpi4py import MPI
comm = MPI.COMM_WORLD

# assuming order in the allgather is the same
coords = comm.allgather(ycoord)
numerical = comm.allgather(numeric)

rank = uw.rank()

if rank == 0 and make_graphs:
    # build matplot lib graph of result only on proc 0

    # 1st build exact solution hiRes
    big = np.linspace(y0,y1)
    cool = analyticT(big)

    %matplotlib inline
    import matplotlib.pyplot as pyplot
    import matplotlib.pylab as pylab
    pylab.rcParams[ 'figure.figsize'] = 12, 6
    pyplot.plot(coords, numerical, 'o', color = 'black', label='numerical') 
    pyplot.plot(big, cool, color = 'red', label="exact") 
    pyplot.xlabel('Y coords')
    pyplot.ylabel('Temperature')
    pyplot.show()

if rank == 0:
    threshold = 1.0e-2
    yspot = y0+1.0e-2
    print "Numerical flux at y = " ,yspot,"is", tField.fn_gradient.evaluate([0.2,yspot])[0][1]
    print "Exact flux at y=" ,yspot,"is", analytic_dT_dy(yspot)
    print "Abs. error ", abserr, " magnitude (will change in parallel due to shadow space) ", mag
    print "Rel. error ", relerr
    if relerr > threshold:
        raise RuntimeError("The numerical solution is outside the error threshold of the analytic solution." \
                           "The Relative error was ", relerr," the threshold is ", threshold)  